In [ ]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import transformers
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
!pip install diffusers transformers accelerate flask flask_cors pyngrok dominate

In [ ]:
!unzip -o /content/drive/MyDrive/music_model_1.zip -d /content/MusicClassifier

Archive:  /content/drive/MyDrive/music_model_1.zip
  inflating: /content/MusicClassifier/config.json  
  inflating: /content/MusicClassifier/tokenizer_config.json  
  inflating: /content/MusicClassifier/vocab.txt  
  inflating: /content/MusicClassifier/special_tokens_map.json  
  inflating: /content/MusicClassifier/model.safetensors  


In [ ]:
!unzip -o /content/drive/MyDrive/finetuned-crd3-flan.zip -d /content/flan-crd3

Archive:  /content/drive/MyDrive/finetuned-crd3-flan.zip
  inflating: /content/flan-crd3/model.safetensors  
  inflating: /content/flan-crd3/spiece.model  
  inflating: /content/flan-crd3/generation_config.json  
  inflating: /content/flan-crd3/tokenizer_config.json  
  inflating: /content/flan-crd3/config.json  
  inflating: /content/flan-crd3/tokenizer.json  
  inflating: /content/flan-crd3/special_tokens_map.json  


In [ ]:
from flask import Flask, request, jsonify
from flask_cors import CORS
from pyngrok import ngrok
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import AutoModelForSequenceClassification
from transformers import pipeline
from diffusers import DiffusionPipeline
from diffusers import FluxPipeline
from PIL import Image
from io import BytesIO
import base64
import subprocess
from transformers import AutoModelForSeq2SeqLM
import re

ngrok.set_auth_token("2w8aKWMARto60g03g5GSYTGkufs_5oKDxCGcfRhU7NnvSNeCy")

model_name = "teknium/OpenHermes-2.5-Mistral-7B"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto"
)

saved_pictures = 0
text_file = "./prompts"

model_name_summarization = "./flan-crd3"
tokenizer_summarization = AutoTokenizer.from_pretrained(model_name_summarization)
model_name_summarization = AutoModelForSeq2SeqLM.from_pretrained(model_name_summarization, torch_dtype=torch.float16, device_map="auto")

pipe = DiffusionPipeline.from_pretrained(
    "darkstorm2150/Protogen_x5.8_Official_Release",
    torch_dtype=torch.float16
)
pipe = pipe.to("cuda")

model_music = AutoModelForSequenceClassification.from_pretrained(
    "/content/MusicClassifier",
    torch_dtype=torch.float16,
    device_map="auto"
)
tokenizer_music = AutoTokenizer.from_pretrained(
    "/content/MusicClassifier"
)
pipe_music = pipeline("text-classification", model=model_music, tokenizer=tokenizer_music)

def clean_prompt_output(text):
    text = text.strip()
    sentences = re.split(r'(?<=[.!?])\s+', text)
    unique = []
    for s in sentences:
        if not unique or s.strip() != unique[-1]:
            unique.append(s.strip())
    for s in unique:
        if len(s.split()) >= 5:
            return s
    return unique[0] if unique else text


def chat_with_model(prompt, max_new_tokens=256):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    input_length = inputs["input_ids"].shape[1]

    with torch.no_grad():
        output = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            temperature=0.7,
            top_p=0.9,
            eos_token_id=tokenizer.eos_token_id,
            pad_token_id=tokenizer.eos_token_id
        )

    new_tokens = output[0][input_length:]
    return tokenizer.decode(new_tokens, skip_special_tokens=True).strip()


app = Flask(__name__)
CORS(app)

@app.route("/chat", methods=["POST"])
def chat():
    data = request.get_json()
    prompt = data.get("prompt", "")
    print(f"Data in chat = {data}")
    print(f"[Prompt primit] {prompt}")
    result = chat_with_model(prompt)
    print(f"Result in chat = {result}")
    return jsonify({"response": result})

@app.route("/generate_image", methods=["POST"])
def generate_image():
    data = request.get_json()
    prompt = data.get("prompt", "")

    global saved_pictures

    if not prompt:
        return jsonify({"error": "Missing 'prompt' in request."}), 400

    image = pipe(prompt, guidance_scale=7.5).images[0]
    image.save("./drive/MyDrive/OdoPixels/pytorch-CycleGAN-and-pix2pix/datasets/pix2pix_dataset/testA/generated_image.png")

    output = subprocess.run([
    "python", "/content/drive/MyDrive/OdoPixels/pytorch-CycleGAN-and-pix2pix/test.py",
    "--dataroot", "/content/drive/MyDrive/OdoPixels/pytorch-CycleGAN-and-pix2pix/datasets/pix2pix_dataset",
    "--checkpoints_dir", "/content/drive/MyDrive/OdoPixels/pytorch-CycleGAN-and-pix2pix/checkpoints",
    "--results_dir", "/content/drive/MyDrive/OdoPixels/pytorch-CycleGAN-and-pix2pix/results",
    "--name", "pix2pix_cycle_gan",
    "--model", "test",
    "--epoch", "latest",
    "--no_dropout",
    "--num_test", "1"
    ])

    print(output)

    pixelated_image = Image.open("./drive/MyDrive/OdoPixels/pytorch-CycleGAN-and-pix2pix/results/pix2pix_cycle_gan/test_latest/images/generated_image_fake.png")

    buffered = BytesIO()
    pixelated_image.save(buffered, format="PNG")
    pixelated_image.save(f"image_{saved_pictures}.png")
    saved_pictures += 1
    img_str = base64.b64encode(buffered.getvalue()).decode("utf-8")

    return jsonify({"image_base64": img_str})

@app.route("/generate_prompt", methods=["POST"])
def generate_prompt():
    data = request.get_json()
    print(f"Data: {data}")
    input = data.get("prompt", "")

    print(f"Input: {input}")

    global text_file

    if not input:
        return jsonify({"error": "Missing 'prompt' in request."}), 400

    with open(text_file, "a") as f:
        f.write(f"Original story: {input}\n")

    inputs = tokenizer_summarization(input, return_tensors="pt", truncation=False)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    outputs = model_name_summarization.generate(**inputs, max_new_tokens=60)
    print(f"Tokenizer output: {tokenizer_summarization.decode(outputs[0], skip_special_tokens=True)}")
    output = clean_prompt_output(tokenizer_summarization.decode(outputs[0], skip_special_tokens=True))

    print(f"Summary:\n{output}")

    token_count = 69

    with open(text_file, "a") as f:
        f.write(f"Summary: {output}\n")

    return jsonify({
        "image_prompt": output
    })

@app.route("/classify_music", methods=["POST"])
def classify_music():
    data = request.get_json()
    input = data.get("prompt", "")

    if not input:
        return jsonify({"error": "Missing 'prompt' in request."}), 400

    classification = pipe_music(input)
    emotion = classification[0]["label"]
    return jsonify({"response": emotion})





public_url = ngrok.connect(5000)
print("🔗 Server live:", public_url)

app.run(port=5000)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.60k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/51.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/101 [00:00<?, ?B/s]

...

INFO:werkzeug:127.0.0.1 - - [26/May/2025 13:29:17] "POST /generate_prompt HTTP/1.1" 200 -


Tokenizer output: He tied one end of the rope to a sturdy tree on the bank and threw the other end across the river. He then climbed down the tree and used the rope to swing across the river.
Summary:
He tied one end of the rope to a sturdy tree on the bank and threw the other end across the river.


  0%|          | 0/50 [00:00<?, ?it/s]

INFO:werkzeug:127.0.0.1 - - [26/May/2025 13:29:18] "POST /classify_music HTTP/1.1" 200 -
Potential NSFW content was detected in one or more images. A black image will be returned instead. Try again with a different prompt and/or seed.
INFO:werkzeug:127.0.0.1 - - [26/May/2025 13:29:25] "POST /generate_image HTTP/1.1" 200 -


CompletedProcess(args=['python', '/content/drive/MyDrive/OdoPixels/pytorch-CycleGAN-and-pix2pix/test.py', '--dataroot', '/content/drive/MyDrive/OdoPixels/pytorch-CycleGAN-and-pix2pix/datasets/pix2pix_dataset', '--checkpoints_dir', '/content/drive/MyDrive/OdoPixels/pytorch-CycleGAN-and-pix2pix/checkpoints', '--results_dir', '/content/drive/MyDrive/OdoPixels/pytorch-CycleGAN-and-pix2pix/results', '--name', 'pix2pix_cycle_gan', '--model', 'test', '--epoch', 'latest', '--no_dropout', '--num_test', '1'], returncode=0)
